In [2]:
import json
import pandas as pd
from tensorflow.keras.layers import (
    Bidirectional,
    GRU,
    Dropout,
    Dense,
    GRU,
    Dense,
)


from tensorflow.keras.models import Sequential


def custom_scale(data):
    """Chuẩn hóa dữ liệu về khoảng 0.25 đến 1."""
    return data / 36000


def custom_inverse_scale(data):
    """Chuyển dữ liệu đã chuẩn hóa trở lại quy mô ban đầu."""
    return data * 36000


# Đọc dữ liệu từ tập tin JSON
file_path = "hsg.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(
    {
        "Date": pd.to_datetime(data["t"], unit="s"),
        "Open": data["o"],
        "High": data["h"],
        "Low": data["l"],
        "Close": data["c"],
        "Volume": data["v"],
    }
)

# Sử dụng các giá trị hiện tại để dự đoán giá 'Open' của ngày tiếp theo
df["Next Open"] = df["Open"].shift(-1)
df.dropna(inplace=True)  # Loại bỏ bất kỳ hàng nào chứa NaN

# Chia dữ liệu thành features (X) và target (y)
X = df[["High", "Low", "Close", "Volume"]].values
y = df["Next Open"].values

# Chuẩn hóa dữ liệu
X_scaled = custom_scale(X)
y_scaled = custom_scale(y)

# Thêm một chiều vào X để phù hợp với đầu vào của GRU
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))


In [27]:
# Xây dựng mô hình GRU
model = Sequential(
    [
        Bidirectional(
            GRU(
                units=128,
                return_sequences=True,
                input_shape=(X_scaled.shape[1], X_scaled.shape[2]),
            )
        ),
        Dropout(0.2),
        Bidirectional(GRU(units=128)),
        Dropout(0.2),
        Dense(64, kernel_initializer="uniform", activation="relu"),
        Dense(1, kernel_initializer="uniform", activation="linear"),
        Dense(1),
    ]
)

# Biên dịch mô hình
model.compile(optimizer="adam", loss="mean_squared_error")

In [28]:
# Huấn luyện mô hình trên toàn bộ dữ liệu
model.fit(X_scaled, y_scaled, epochs=256, batch_size=2048, verbose=1)

# Lưu mô hình
# model.save("gru_model.h5")
predicted_next_open_scaled = model.predict(
    X_scaled[-1].reshape(1, 1, X_scaled.shape[2])
)
predicted_next_open = custom_inverse_scale(predicted_next_open_scaled)
print(
    f"Giá mở cửa dự đoán cho ngày tiếp theo (đã đưa về quy mô ban đầu): {predicted_next_open[0][0]}"
)

Epoch 1/256
3/3 [==============================] - 4s 158ms/step - loss: 1.1687
Epoch 2/256
3/3 [==============================] - 0s 42ms/step - loss: 0.8803
Epoch 3/256
3/3 [==============================] - 0s 40ms/step - loss: 0.5897
Epoch 4/256
3/3 [==============================] - 0s 33ms/step - loss: 0.3560
Epoch 5/256
3/3 [==============================] - 0s 26ms/step - loss: 0.2914
Epoch 6/256
3/3 [==============================] - 0s 34ms/step - loss: 0.3447
Epoch 7/256
3/3 [==============================] - 0s 29ms/step - loss: 0.3196
Epoch 8/256
3/3 [==============================] - 0s 28ms/step - loss: 0.2884
Epoch 9/256
3/3 [==============================] - 0s 31ms/step - loss: 0.3129
Epoch 10/256
3/3 [==============================] - 0s 27ms/step - loss: 0.3061
Epoch 11/256
3/3 [==============================] - 0s 56ms/step - loss: 0.3044
Epoch 12/256
3/3 [==============================] - 0s 38ms/step - loss: 0.2760
Epoch 13/256
3/3 [==============================

In [13]:
# model.save("final_2.h5")

/Users/khanhle/miniforge3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
import json
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
# Đọc lại dữ liệu từ tập tin JSON
file_path = "hsg.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(
    {
        "Date": pd.to_datetime(data["t"], unit="s"),
        "Open": data["o"],
        "High": data["h"],
        "Low": data["l"],
        "Close": data["c"],
        "Volume": data["v"],
    }
)

# Thêm dữ liệu mới vào cuối
# new_data = [1711065600, 22850.0, 23450.0, 22700.0, 23250.0, 31322600.0]  
# df.loc[len(df)] = new_data

# Sử dụng các giá trị hiện tại để dự đoán giá 'Open' của ngày tiếp theo
df["Next Open"] = df["Open"].shift(-1)

# Loại bỏ hàng chứa NaN (hàng cuối cùng)
df.dropna(inplace=True)

# Chia dữ liệu thành features (X) và target (y)
X = df[["High", "Low", "Close", "Volume"]].values
y = df["Next Open"].values

# Chuẩn hóa dữ liệu
X_scaled = custom_scale(X)

# Tải lại mô hình đã lưu
model = load_model("final_2.h5")

# Dự đoán giá mở cửa cho ngày tiếp theo
predicted_next_open_scaled = model.predict(X_scaled[-1].reshape(1, 1, X_scaled.shape[1]))
predicted_next_open = custom_inverse_scale(predicted_next_open_scaled)

print(
    f"Giá mở cửa dự đoán cho ngày tiếp theo (đã đưa về quy mô ban đầu): {predicted_next_open[0][0]}"
)

2024-03-25 22:46:13.007548: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-25 22:46:13.007571: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-25 22:46:13.007582: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-25 22:46:13.007610: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-25 22:46:13.007624: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-03-25 22:46:14.027523: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 635ms/step
Giá mở cửa dự đoán cho ngày tiếp theo (đã đưa về quy mô ban đầu): 23842.384765625
